In [0]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
import os

print("tensorflow version : ", tf.__version__)
print("tensorflow_hub version : ", hub.__version__)

tensorflow version :  1.15.0
tensorflow_hub version :  0.7.0


In [0]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.8MB/s 


In [0]:
#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
OUTPUT_DIR = '/content/bert'

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: /content/bert *****


In [0]:
#train = pd.read_csv("/content/text_emotion.csv")
train = pd.read_excel("/content/text_emotion.xlsx")
train.shape

(24442, 2)

In [0]:
train.head()

,content,sentiment
0,tiffanylue i know i was listenin to bad habit...,2
1,wants to hang out with friends SOON,5
2,Got the news,5
3,"Wondering why I'm awake at 7am,writing a new s...",4
4,It is so annoying when she starts typing on he...,1


In [0]:
train["content"] = train["content"].astype(str)

In [0]:
train['content']=train['content'].apply(lambda x:x.replace(r"(http|@)\S+", ""))
train['content']=train['content'].apply(lambda x:x.replace(r".", ""))

In [0]:
from sklearn.model_selection import train_test_split

train, val =  train_test_split(train, test_size = 0.25, random_state = 100)

In [0]:
train["sentiment"].value_counts()

4    5682
5    4717
2    4599
3    2211
1    1122
Name: sentiment, dtype: int64

In [0]:
print("Training Set Shape :", train.shape)
print("Validation Set Shape :", val.shape)
#print("Test Set Shape :", test.shape)

Training Set Shape : (18331, 2)
Validation Set Shape : (6111, 2)


In [0]:
DATA_COLUMN = 'content'
LABEL_COLUMN = 'sentiment'
# The list containing all the classes (train['SECTION'].unique())
label_list = [1, 2, 3, 4,5]

In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
train_InputExamples

865      <bert.run_classifier.InputExample object at 0x...
23992    <bert.run_classifier.InputExample object at 0x...
17261    <bert.run_classifier.InputExample object at 0x...
15028    <bert.run_classifier.InputExample object at 0x...
12382    <bert.run_classifier.InputExample object at 0x...
                               ...                        
12119    <bert.run_classifier.InputExample object at 0x...
24423    <bert.run_classifier.InputExample object at 0x...
14147    <bert.run_classifier.InputExample object at 0x...
23320    <bert.run_classifier.InputExample object at 0x...
5640     <bert.run_classifier.InputExample object at 0x...
Length: 18331, dtype: object

In [0]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[15300].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[15300].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[15300].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[15300].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  tracekase GO TO BED

__________
Row 0 - text_b of training set :  None

__________
Row 0 - label of training set :  1


In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
#Here is what the tokenised sample of the first training set observation looks like
print(tokenizer.tokenize(train_InputExamples.iloc[0].text_a))

['che', '##lle', '##mar', '##ie', '##e', 'i', 'have', 'my', '83', '##0', 'class', 'fuck', '##in', 'school', 'what', 'time', 'you', 'work', 'wed', '?', 'if', 'jade', 'comes', 'im', '##a', 'have', 'to', 'get', 'her', 'in', 'antioch', 'at', 'amtrak']


In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 18331


INFO:tensorflow:Writing example 0 of 18331


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 4 (id = 3)


INFO:tensorflow:label: 4 (id = 3)


In [0]:
#Example on first observation in the training set
print("Sentence : ", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-"*30)
print("Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("Input Masks : ", train_features[0].input_mask)
print("-"*30)
print("Segment IDs : ", train_features[0].segment_ids)

Sentence :  chellemariee I have my 830 class  fuckin school what time you work wed? if jade comes ima have to get her in antioch at amtrak
------------------------------
Tokens :  ['che', '##lle', '##mar', '##ie', '##e', 'i', 'have', 'my', '83', '##0', 'class', 'fuck', '##in', 'school', 'what', 'time', 'you', 'work', 'wed', '?', 'if', 'jade', 'comes', 'im', '##a', 'have', 'to', 'get', 'her', 'in', 'antioch', 'at', 'amtrak']
------------------------------
Input IDs :  [101, 18178, 6216, 7849, 2666, 2063, 1045, 2031, 2026, 6640, 2692, 2465, 6616, 2378, 2082, 2054, 2051, 2017, 2147, 21981, 1029, 2065, 12323, 3310, 10047, 2050, 2031, 2000, 2131, 2014, 1999, 19078, 2012, 20208, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Inp

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
#A function that adapts our model to work for training, evaluation, and prediction.

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 100
SAVE_SUMMARY_STEPS = 10

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
print(num_train_steps)

1718


In [0]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/bert', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f005f277470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/bert', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f005f277470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
#Training the model
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 1000 into /content/bert/model.ckpt.


INFO:tensorflow:global_step/sec: 0.0239545


INFO:tensorflow:global_step/sec: 0.0239545


INFO:tensorflow:loss = 0.870662, step = 1001 (4174.587 sec)


INFO:tensorflow:loss = 0.870662, step = 1001 (4174.587 sec)


In [0]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=val_input_fn, steps=None)

In [0]:
"""delighted: 5
happy: 4
neutral: 3
disappointed: 2
frustrated: 1"""

# A method to get predictions
def getPrediction(in_sentences):
  #A list to map the actual labels to the predictions
  labels = ["delighted",	"happy",	"neutral",	"disappointed",	"frustrated"]

  #Transforming the test data into BERT accepted form
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  
  #Creating input features for Test data
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

  #Predicting the classes 
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'],prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = list(val['content'])

In [0]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 5170


INFO:tensorflow:Writing example 0 of 5170


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from /content/bert/model.ckpt-30


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
predictions

[('wirbelwind1111 I just LOVE it.',
  array([-2.6905646, -2.3361478, -2.9611049, -1.9936782, -2.50997  ,
         -2.4876127, -1.942195 , -2.7414498, -2.9335477, -2.5429692,
         -3.281128 , -2.6289628, -3.3908913], dtype=float32),
  6,
  'love'),
 ("I'm watching The episode 3 Of JONAS OMJ",
  array([-2.7403712, -2.5682092, -2.8525171, -1.9114983, -2.8676891,
         -2.440689 , -1.8736012, -2.8657262, -2.565339 , -2.6707573,
         -3.0974302, -2.73469  , -3.155693 ], dtype=float32),
  6,
  'love'),
 ('Rocking out to a Shania Twain marathon',
  array([-2.8802218, -2.4720407, -2.750665 , -2.0547478, -2.7174478,
         -2.708696 , -1.8960803, -2.63577  , -2.657163 , -2.2446208,
         -3.1776283, -2.883865 , -3.237777 ], dtype=float32),
  6,
  'love'),
 ('qi_tah oh no.  *hugs*',
  array([-2.7873917, -2.3707407, -2.9872916, -2.044106 , -2.7917054,
         -2.4644802, -2.0503383, -2.571628 , -2.644977 , -2.3138015,
         -3.1423779, -2.8403273, -3.166831 ], dtype=float32),


In [0]:
df = pd.DataFrame(predictions)
df.to_csv('output_bert.csv', index=False)

In [0]:
from sklearn import metrics
print(metrics.classification_report(test["SECTION"], df[2]))

              precision    recall  f1-score   support

           0       0.33      0.42      0.37       214
           1       0.67      0.29      0.40       915
           2       0.49      0.81      0.61       493
           3       0.67      0.75      0.71      1285
           4       0.79      0.78      0.79       591

    accuracy                           0.62      3498
   macro avg       0.59      0.61      0.57      3498
weighted avg       0.65      0.62      0.61      3498



In [0]:
enc_labels = []
act_labels = []
for i in range(len(predictions)):
  enc_labels.append(predictions[i][2])
  act_labels.append(predictions[i][3])
  story.append(predictions[i])

In [0]:
#Classifying random sentences
tests = getPrediction(["because it’s the ultimate driving machine!!",
"i didn't like the financing process.",
"in the process of that we discovered a financial anomaly that was agreed upon verbally twice by our sales person but did not make its way into our sales documentation."])

INFO:tensorflow:Writing example 0 of 3


INFO:tensorflow:Writing example 0 of 3


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from /content/bert/model.ckpt-30


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
tests

[('because it’s the ultimate driving machine!!',
  array([-3.0360065, -2.1916754, -2.825334 , -2.044458 , -2.9040942,
         -2.2852817, -2.3617618, -2.6351657, -2.4035568, -2.4552956,
         -3.2081861, -2.7129292, -3.0792356], dtype=float32),
  3,
  'neutral'),
 ("i didn't like the financing process.",
  array([-2.7179046, -2.3504322, -2.840351 , -2.0822709, -2.8967247,
         -2.201397 , -2.3391757, -2.6674876, -2.6255329, -2.3236697,
         -2.994443 , -2.7731175, -3.1923518], dtype=float32),
  3,
  'neutral'),
 ('in the process of that we discovered a financial anomaly that was agreed upon verbally twice by our sales person but did not make its way into our sales documentation.',
  array([-2.467403 , -2.4079738, -3.0670233, -2.1521125, -2.867617 ,
         -2.384957 , -2.303887 , -2.7507935, -2.5461812, -2.2795525,
         -3.0494087, -2.676087 , -2.9413176], dtype=float32),
  3,
  'neutral')]

In [0]:
full_raw_emotion = pd.read_excel("/content/full_raw.xlsx")

In [0]:
full_raw_emotion.shape

(10000, 2)

In [0]:
full_raw_emotion['answer_nps_comment_split']=full_raw_emotion['answer_nps_comment_split'].apply(lambda x:x.replace(r"\", ""))
full_raw_emotion['answer_nps_comment_split']=full_raw_emotion['answer_nps_comment_split'].apply(lambda x:x.replace(r".", ""))
full_raw_emotion['answer_nps_comment_split']=full_raw_emotion['answer_nps_comment_split'].apply(lambda x:x.replace(r",", ""))
full_raw_emotion['answer_nps_comment_split']=full_raw_emotion['answer_nps_comment_split'].apply(lambda x:x.replace(r"(", ""))

In [0]:
full_raw_emotion_pred=getPrediction(full_raw_emotion['answer_nps_comment_split'])

INFO:tensorflow:Writing example 0 of 10000


INFO:tensorflow:Writing example 0 of 10000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from /content/bert/model.ckpt-367


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
pd.DataFrame(full_raw_emotion_pred).to_excel("full_raw_emotions.xlsx",header=False,index=False)

In [0]:
from google.colab import files
files.download("full_raw_emotions.xlsx")

MessageError: ignored

In [0]:
tf.reset_default_graph()
    
with tf.Session() as session:
    # Restore variables and model
    saver = tf.train.import_meta_graph("/content/bert/model.ckpt-30.meta")
    saver.restore(session, tf.train.latest_checkpoint("/content/bert/"))
    print("Model restored.")

INFO:tensorflow:Restoring parameters from /content/bert/model.ckpt-30


INFO:tensorflow:Restoring parameters from /content/bert/model.ckpt-30


Model restored.


In [0]:
graph = tf.get_default_graph()
input_ids = graph.get_tensor_by_name('module/input_ids:0')
input_mask = graph.get_tensor_by_name('module/input_mask:0')
segment_ids = graph.get_tensor_by_name('module/segment_ids:0')
labels = graph.get_tensor_by_name('save_2/Assign_314:0')

In [0]:
prediction = session.run(labels,feed_dict={input_features, MAX_SEQ_LENGTH, False, False})
print(prediction)

NameError: ignored